In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.nn as nn
import torch

from tqdm import tqdm
import numpy as np
import os

In [ ]:
# set seeds for reproducibility
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True

In [ ]:
# set URL to download data
translation_data_url = "https://raw.githubusercontent.com/cetinsamet/data-science/main/data/translation/EN2TR.txt"
# get data filename
translation_data_filename = os.path.basename(translation_data_url)

# download the data in local if it doesn't already exist
if not os.path.exists(translation_data_filename):
    try:
        !wget $translation_data_url
        print("File is succesfully downloaded.")
    except Exception as e:
        print(f"Could not download the book from {translation_data_url}")
        print(e)
else:
    print("File has already been downloaded.")

In [ ]:
# define hyperparameters
N_EPOCHS = 30
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
EMBED_SIZE = 256
HIDDEN_SIZE = 512
NUM_LAYERS = 1
TEST_SIZE = 0.2
SAMPLE_RATIO = 1.0
EARLY_STOPPING_STEP_SIZE = 5
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BEST_ENCODER_FILEPATH = 'best_encoder.pt'
BEST_DECODER_FILEPATH = 'best_decoder.pt'

DEVICE

In [ ]:
def load_data(fp, sep='\t', remove_last_char=False, drop_first=False):
    with open(fp, mode='r') as infile:
        sentences = [row.strip().split(sep) for idx, row in enumerate(infile)]
        if drop_first:
            sentences = sentences[1:]
    sentences_src, sentences_tgt = zip(*sentences)
    if remove_last_char:
        sentences_src = [sentence[:-1] for sentence in sentences_src]
        sentences_tgt = [sentence[:-1] for sentence in sentences_tgt]
    return sentences_src, sentences_tgt

In [ ]:
# load all sentences for both source and target languages
all_sentences_src, all_sentences_tgt = load_data(translation_data_filename, remove_last_char=True)
assert len(all_sentences_src) == len(all_sentences_tgt)  # sanity check

# get the number of all sentences
n_all_sentences = len(all_sentences_src)
print(f"There are {n_all_sentences} in total")

### sample sentences with a SAMPLE_RATIO to get a portion to use in training
# get the number of (sampled) sentences
n_sentences = int(n_all_sentences * SAMPLE_RATIO)
# get the sampled source and target sentences
sentences_src, sentences_tgt = all_sentences_src[:n_sentences], all_sentences_tgt[:n_sentences]
print(f"There are {n_sentences} in total after sampling %{SAMPLE_RATIO*100} of all sentences")

# display some example translation from the data
print('\nExamples:')
for idx in np.random.randint(low=0, high=n_sentences, size=5):
    print(f"{sentences_src[idx]} <---> {sentences_tgt[idx]}")

In [ ]:
### split sentences into train and test sets
# set the number of test sentences
n_test = int(n_sentences * TEST_SIZE)
# randomly select test sentence indices
test_indices = np.random.randint(low=0, high=n_sentences, size=n_test)

# split sentences into train and test, source and target language sets
train_data_src, train_data_tgt = [], []
test_data_src, test_data_tgt = [], []
for i in tqdm(range(n_sentences)):
    if i in test_indices:
        test_data_src.append(sentences_src[i])
        test_data_tgt.append(sentences_tgt[i])
    else:
        train_data_src.append(sentences_src[i])
        train_data_tgt.append(sentences_tgt[i])

# get the number of train sentences
n_train = len(train_data_src)
print(f"There {n_train} sentences in train data.")

# get the number test train sentences
n_test = len(test_data_src)
print(f"There {n_test} sentences in test data.")

In [ ]:
# define function to tokenize sentences (in a very simple way)
def tokenize(sentences):
    return [el for sentence in sentences for el in sentence.lower().split()]

In [ ]:
# define special tokens (we won't use <UNK> but it's a good practise to have it)
special_tokens = ['<PAD>', '<SOS>', '<EOS>', '<UNK>']

# define source and target language vocabularies by tokenizing sentences
# (and add special tokens to both vocabularys)
vocab_src = special_tokens + sorted(list(set(tokenize(sentences_src))))
vocab_tgt = special_tokens + sorted(list(set(tokenize(sentences_tgt))))

# get the source and target languages' vocabulary sizes
vocab_src_size = len(vocab_src)
vocab_tgt_size = len(vocab_tgt)

print(f"Vocabulary (SRC) size = {vocab_src_size}")
print(f"Vocabulary (TGT) size = {vocab_tgt_size}")
print("--------------\n")

In [ ]:
# define the mappings (as dictionary) of 
# 'a token (from the vocab) to a unique ID' ---> 'char2int'
# and
# 'a unique ID to a token (from the vocab)'  ---> 'int2char'
# for both source and target languages
char2int_src = {c: idx for idx, c in enumerate(vocab_src)}
int2char_src = {idx: c for idx, c in enumerate(vocab_src)}

char2int_tgt = {c: idx for idx, c in enumerate(vocab_tgt)}
int2char_tgt = {idx: c for idx, c in enumerate(vocab_tgt)}

# define the encode() that encodes/converts a token to a unique ID 
def encode(character, char2int):
    return char2int[character]

# define the encode() that decodes/converts back a unique ID to a token
def decode(integer, int2char):
    return int2char[integer]

# show an example of how encode() and decode() operate
sentence_src = 'What time is it?'
sentence_src_encoded = [encode(token, char2int_src) for token in tokenize([sentence_src])]
print(f"'{sentence_src}' is encoded as {sentence_src_encoded}.")

sentence_src_encoded_decoded = [decode(token, int2char_src) for token in sentence_src_encoded]
print(f"'{sentence_src_encoded}' is encoded as '{' '.join(sentence_src_encoded_decoded)}'.")
print("--------------\n")

In [ ]:
# define the LanguageDataset as an instance of torch.nn.Dataset
class LanguageDataset(Dataset):
    def __init__(self, sentences_src, sentences_tgt, vocab_src, vocab_tgt):
        super().__init__()
        self.sentences_src = sentences_src
        self.sentences_tgt = sentences_tgt
        self.vocab_src = vocab_src
        self.vocab_tgt = vocab_tgt

    def __len__(self):
        return len(self.sentences_src)

    def __getitem__(self, idx):
        x_src = torch.tensor(
            [encode(token, char2int_src) for token in tokenize([self.sentences_src[idx]])], 
            dtype=torch.long
        )
        x_tgt = torch.tensor(
            [encode(token, char2int_tgt) for token in tokenize([self.sentences_tgt[idx]])],
            dtype=torch.long
        )
        return x_src, x_tgt

In [ ]:
def custom_pad_sequence(data):
    global char2int_src, char2int_tgt
    x_src, x_tgt = zip(*data)
    
    # pad 0 (which is uniqueID of the <PAD> token since we designed the vocabularies this way) 
    # to make each sampled sentences have the same sequence length to make batch training possible
    x_src = pad_sequence(x_src, batch_first=True)
    x_tgt = pad_sequence(x_tgt, batch_first=True)
    
    # add <EOS> (End Of Sentence) token at the end of each sampled source and target sentences
    eos_src, eos_tgt = encode('<EOS>', char2int_src), encode('<EOS>', char2int_tgt)
    x_src = torch.cat((x_src, torch.full((len(x_src), 1), eos_src)), dim=1)
    x_tgt = torch.cat((x_tgt, torch.full((len(x_tgt), 1), eos_tgt)), dim=1)
    return x_src, x_tgt
    
# initialize train and test datasets
train_dset = LanguageDataset(train_data_src, train_data_tgt, vocab_src, vocab_tgt)
test_dset = LanguageDataset(test_data_src, test_data_tgt, vocab_src, vocab_tgt)

# initialize train and test iterators
train_iterator = DataLoader(train_dset, BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=custom_pad_sequence, pin_memory=True)
test_iterator = DataLoader(test_dset, BATCH_SIZE, shuffle=False, collate_fn=custom_pad_sequence, pin_memory=True)

In [ ]:
# define Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers):
        super().__init__()  
        self.embedding_layer = nn.Embedding(input_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x = torch.flip(x, dims=[1])
        x = self.embedding_layer(x)
        x, (hidden_state, cell_state) = self.lstm(x)
        return x, hidden_state, cell_state

In [ ]:
# define Decoder
class Decoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers):
        super().__init__()  
        self.embedding_layer = nn.Embedding(input_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x, hidden_state, cell_state):
        x = self.embedding_layer(x)
        x, (hidden_state, cell_state) = self.lstm(x, (hidden_state, cell_state))
        x = self.fc(x)
        return x, hidden_state, cell_state

In [ ]:
def translate(sentence, encoder, decoder, max_len=10):
    # global variables
    global DEVICE, char2int_src, char2int_tgt, int2char_tgt
    # set encoder and decoder to evaluation mode
    encoder.eval(); decoder.eval()
    # get <SOS> token ID for decoder's first forward pass and 
    # get <EOS> token ID to add it to the end of the sentence that will be translated
    sos_tgt = encode('<SOS>', char2int_tgt)
    eos_src = encode('<EOS>', char2int_src)
    
    with torch.inference_mode():
        # encode the input sentence
        x = torch.tensor([encode(token, char2int_src) for token in tokenize([sentence])]).view(1, -1)
        # add <EOS> token at the end of the input sentence
        x = torch.cat((x, torch.full((1, 1), eos_src)), dim=1).long().to(DEVICE)
        
        # encoder forward pass
        _, encoder_hidden_state, encoder_cell_state = encoder(x)        
            
        # create <SOS> token as the first input token for the decoder pass
        next_token = torch.full((1, 1), sos_tgt, dtype=torch.long).to(DEVICE)
        # rename encoder's hidden and cell state tensors to ease the decoder loop below
        decoder_hidden_state = encoder_hidden_state
        decoder_call_state = encoder_cell_state
            
        translation = []
        # generate max_len character at most
        for _ in range(max_len):
            # decoder forward pass
            logits, decoder_hidden_state, decoder_call_state = decoder(next_token, decoder_hidden_state, decoder_call_state) 
            # get the next token
            next_token = torch.argmax(logits.view(-1))
            # get the next character
            next_char = decode(next_token.item(), int2char_tgt)
            
            # if <EOS> is predicted as the next char, break the generation loop
            if next_char == '<EOS>':
                break
            else:
                translation.append(next_char) 
            
            next_token = next_token.view(1, -1)

        translated_sentence = ' '.join(translation)
        return translated_sentence

In [ ]:
def evaluate(encoder, decoder, iterator):
    # global variables
    global DEVICE, char2int_tgt
    # set encoder and decoder to evaluation mode
    encoder.eval(); decoder.eval()
    # initialize the loss sum
    loss_sum = 0.0
    
    with torch.inference_mode():
        # iterate over batches
        for iter_idx, (x_src, x_tgt) in tqdm(enumerate(iterator), total=len(iterator)):
            # carry tensors to available device
            x_src, x_tgt = x_src.to(DEVICE), x_tgt.to(DEVICE)
            # encoder forward pass
            _, encoder_hidden_state, encoder_cell_state = encoder(x_src)
            
            # create a tensor only containing <SOS> token as input for the decoder
            batch_size = len(x_src)  # get current batch size
            sos_tgt = encode('<SOS>', char2int_tgt)
            decoder_input = torch.full((batch_size, 1), sos_tgt, dtype=torch.long).to(DEVICE)
            # rename the hidden state and cell state variables for loop below
            decoder_hidden_state, decoder_cell_state = encoder_hidden_state, encoder_cell_state
            
            # initialize a list to store logits computed for each target token
            logits_all = []
            # iterate over target sequence
            for tgt_idx in range(x_tgt.shape[1]):
                # decoder forward pass
                logits, decoder_hidden_state, decoder_cell_state = decoder(decoder_input, decoder_hidden_state, decoder_cell_state)
                # store logits
                logits_all.append(logits)
                # get the next target token as input for the next decoder forward pass
                decoder_input = x_tgt[:,[tgt_idx]]
            
            # stack stored logits and compute loss
            logits_stacked = torch.hstack(logits_all)
            B, S, C = logits_stacked.shape # B: batch size, S: seq length, C: channnel (or embed) size
            loss = F.cross_entropy(logits_stacked.view(B*S, C), x_tgt.view(-1), ignore_index=pad_tgt)
            
            # add batch loss to total loss sum
            loss_sum += loss.item()
        
    # compute avg loss
    loss_avg = loss_sum / len(iterator)
    return loss_avg

In [ ]:
# initialize the encoder
encoder = Encoder(
    input_size=vocab_src_size,
    embed_size=EMBED_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
).to(DEVICE)

# initialize the decoder
decoder = Decoder(
    input_size=vocab_tgt_size,
    embed_size=EMBED_SIZE, 
    hidden_size=HIDDEN_SIZE, 
    num_layers=NUM_LAYERS,
).to(DEVICE)

# initialize encoder and decoder optimizers
optimizer_encoder = torch.optim.AdamW(encoder.parameters(), lr=LEARNING_RATE)
optimizer_decoder = torch.optim.AdamW(decoder.parameters(), lr=LEARNING_RATE)

# init the best test loss as positive inifinity
best_test_loss = float('inf')
best_streak_count = 0

# select some example sentences from test set to translate during training
n_example_sentence = 10
example_sentence_indices = np.random.randint(low=0, high=len(test_data_src), size=n_example_sentence)

# iterate over epochs
for epoch_idx in range(1, N_EPOCHS+1):
    # initialize train loss
    loss_train = 0.0
    # set encoder and decoder to train mode
    encoder.train(); decoder.train()
    
    # iterate over batches
    for iter_idx, (x_src, x_tgt) in tqdm(enumerate(train_iterator), total=len(train_iterator)):
        # carry tensors to available device
        x_src, x_tgt = x_src.to(DEVICE), x_tgt.to(DEVICE)
        # encoder forward pass
        _, encoder_hidden_state, encoder_cell_state = encoder(x_src)
        
        # create a tensor only containing <SOS> token as input for the decoder
        sos_tgt = encode('<SOS>', char2int_tgt)
        decoder_input = torch.full((BATCH_SIZE, 1), sos_tgt, dtype=torch.long).to(DEVICE)
        
        # rename the hidden state and cell state variables for loop below
        decoder_hidden_state, decoder_cell_state = encoder_hidden_state, encoder_cell_state
        
        # initialize a list to store logits computed for each target token
        logits_all = []
        # itearate over target sequence
        for tgt_idx in range(x_tgt.shape[1]):
            # decoder forward pass
            logits, decoder_hidden_state, decoder_cell_state = decoder(decoder_input, decoder_hidden_state, decoder_cell_state)
            # store logits
            logits_all.append(logits)
            # get the next target token as input for the next decoder forward pass
            decoder_input = x_tgt[:,[tgt_idx]]
        
        # stack stored logits and compute loss
        logits_stacked = torch.hstack(logits_all)
        pad_tgt = encode('<PAD>', char2int_tgt)        
        B, S, C = logits_stacked.shape  # B: batch size, S: seq length, C: channnel (or embed) size
        loss = F.cross_entropy(logits_stacked.view(B*S, C), x_tgt.view(-1), ignore_index=pad_tgt)
        
        # update gradients
        optimizer_encoder.zero_grad()
        optimizer_decoder.zero_grad()
        loss.backward()
        optimizer_encoder.step()
        optimizer_decoder.step()
        
        # add batch loss to total loss sum
        loss_train += loss.item()

    # print epoch logs
    loss_train /= len(train_iterator)
    loss_test = evaluate(encoder, decoder, test_iterator)
    print(f"epoch {epoch_idx:02} || train loss = {loss_train:.3f}\ttest loss = {loss_test:.3f}", end='\n\n')
    
    # save the current model as the best model if the current test loss is the least achieved 
    if loss_test < best_test_loss:
        # save the curent encoder and decoder's parameters as the best parameters
        torch.save(encoder.state_dict(), BEST_ENCODER_FILEPATH)
        torch.save(decoder.state_dict(), BEST_DECODER_FILEPATH)
        # replace the best test loss with the current best loss
        best_test_loss = loss_test
        # reset early stoppping counter 
        best_streak_count = 0
        # display info
        print(f'The best model is found and saved. Current best test loss = {best_test_loss:.3f}\n')
        # translate example sentences
        for example_idx in example_sentence_indices:
            sentence = test_data_src[example_idx]
            translation = translate(sentence, encoder, decoder)
            print(f"{sentence} <---> {translation}")
    else:
        # update early stoppping counter 
        best_streak_count += 1

    # check early stopping condition
    if best_streak_count == EARLY_STOPPING_STEP_SIZE:
        print(f"A better model has not been found in the last {EARLY_STOPPING_STEP_SIZE} epochs. Early stopping...")
        break
        
    print("--------------\n")

In [ ]:
# reinitialize encoder
best_encoder = Encoder(
    input_size=vocab_src_size,
    embed_size=EMBED_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
).to(DEVICE)
# load best encoder's parameters
best_encoder.load_state_dict(torch.load(BEST_ENCODER_FILEPATH))

# reinitialize decoder
best_decoder = Decoder(
    input_size=vocab_tgt_size,
    embed_size=EMBED_SIZE, 
    hidden_size=HIDDEN_SIZE, 
    num_layers=NUM_LAYERS,
).to(DEVICE)
# load best decoder's parameters
best_decoder.load_state_dict(torch.load(BEST_DECODER_FILEPATH))
print("Encoder and Decoder are initialized and the best model parameters are loaded.")

In [ ]:
# simple example
sentence = "I like hiking"
translation = translate(sentence, best_encoder, best_decoder)
print(f"{sentence} <---> {translation}")

In [ ]:
# harder example
sentence = test_data_src[-100]
translation = translate(sentence, best_encoder, best_decoder)
print(f"{sentence} <---> {translation}")